In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import os 
import sys
import gget
import xlrd
import textwrap
import nb_util as nb

sys.path.append("../python/")
import utils as ut

In [2]:
data_dir = "../data/raw_data/"

df = []
for f in os.listdir(data_dir):
    if f.endswith('.csv'):
        dataset = f.split("_")[0]
        tmp = pd.read_csv(f"{data_dir}{f}")
        tmp['dataset'] = dataset
        df.append(tmp)
        del tmp

df = pd.concat(df)
print(f"{df.shape=}")
df.head()

df.shape=(1269510, 8)


,gene_name,time_id,tpm,time_point,replicate,hours,control,dataset
0,A1BG,D1_T1R1,0.126512,1,1,-48,control,2018
1,A1BG,D1_T2R1,0.179995,2,1,0,timecourse,2018
2,A1BG,D1_T3R1,0.068018,3,1,8,timecourse,2018
3,A1BG,D2_T1R1,0.104575,1,1,16,timecourse,2018
4,A1BG,D2_T2R1,0.196855,2,1,24,timecourse,2018


# get the genes

In [3]:
gx = df.groupby(['gene_name','dataset'])['tpm'].mean().reset_index(drop=False)
gx = pd.pivot_table(gx, index='gene_name',
                    columns='dataset',
                    values='tpm').reset_index(drop=False)

gx.columns = ['gene_name', '2015_mean_tpm', '2018_mean_tpm']
gx.head()

,gene_name,2015_mean_tpm,2018_mean_tpm
0,A1BG,0.077728,0.148410
1,A1CF,0.000428,0.001135
2,A2M,0.000813,6.078858
3,A2ML1,0.000206,0.001135
4,A3GALT2,0.000000,0.003806


# add kegg pathways

In [4]:
kegg_ids = {
    'hsa03020' : 'RNA_polymerase',
    'hsa03022' : 'basal_TFs',
    'hsa04110' : 'cell_cycle',
    'hsa04710' : 'circadian_rhythm',
    'hsa04713' : 'circadian_entrainment',
}

gene_lists = {}

for k, v in kegg_ids.items():
    gene_set = ut.parseKEGG(k)
    print(f"{v} {len(gene_set)}")
    gx[v] = np.where(gx['gene_name'].isin(gene_set), 1, 0)

gx.head()

RNA_polymerase 34
basal_TFs 44
cell_cycle 157
circadian_rhythm 34
circadian_entrainment 97


,gene_name,2015_mean_tpm,2018_mean_tpm,RNA_polymerase,basal_TFs,cell_cycle,circadian_rhythm,circadian_entrainment
0,A1BG,0.077728,0.148410,0,0,0,0,0
1,A1CF,0.000428,0.001135,0,0,0,0,0
2,A2M,0.000813,6.078858,0,0,0,0,0
3,A2ML1,0.000206,0.001135,0,0,0,0,0
4,A3GALT2,0.000000,0.003806,0,0,0,0,0


# add pangloaDB markers

In [5]:
def getGenes(pdf, cellType, ui_upper=None):
    genes = pdf[pdf['cell type'] == cellType]
    if not ui_upper is None:
        genes = genes[genes['ubiquitousness index'] < ui_upper]
    return genes['official gene symbol'].to_list()

pdfPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/panglaodb/pandb.tsv.gz"
pandDf = pd.read_csv(pdfPath, sep="\t")
fb = getGenes(pandDf, 'Fibroblasts')

# get myogenic genes
mg_cells = ['Myoblasts', 'Myofibroblasts', 'Myocytes']
myo = []

for mg in mg_cells:
    myo += getGenes(pandDf, mg)

myo = list(set(myo))
print(f"{len(myo)=}")


gx['fibroblast_pangloa_db'] = np.where(gx['gene_name'].isin(fb), 1, 0)
gx['myogenic_pangloa_db'] = np.where(gx['gene_name'].isin(myo), 1, 0)
gx.head()

len(myo)=117


,gene_name,2015_mean_tpm,2018_mean_tpm,RNA_polymerase,basal_TFs,cell_cycle,circadian_rhythm,circadian_entrainment,fibroblast_pangloa_db,myogenic_pangloa_db
0,A1BG,0.077728,0.148410,0,0,0,0,0,0,0
1,A1CF,0.000428,0.001135,0,0,0,0,0,0,0
2,A2M,0.000813,6.078858,0,0,0,0,0,0,0
3,A2ML1,0.000206,0.001135,0,0,0,0,0,0,0
4,A3GALT2,0.000000,0.003806,0,0,0,0,0,0,0


# add cell cycle sets

In [6]:
# load some gene sets
g2_genes = nb.g2_genes
s_genes = nb.s_genes
pip_fucci_genes = ['PCNA', 'CDT1', 'GMNN', "CDKN1A", "CDK1", ]

gx['G2_marker_gene'] = np.where(gx['gene_name'].isin(g2_genes), 1, 0)
gx['S_marker_gene'] = np.where(gx['gene_name'].isin(s_genes), 1, 0)
gx['pip_fucci_gene'] = np.where(gx['gene_name'].isin(pip_fucci_genes), 1, 0)

gx.head()


,gene_name,2015_mean_tpm,2018_mean_tpm,RNA_polymerase,basal_TFs,cell_cycle,circadian_rhythm,circadian_entrainment,fibroblast_pangloa_db,myogenic_pangloa_db,G2_marker_gene,S_marker_gene,pip_fucci_gene
0,A1BG,0.077728,0.148410,0,0,0,0,0,0,0,0,0,0
1,A1CF,0.000428,0.001135,0,0,0,0,0,0,0,0,0,0
2,A2M,0.000813,6.078858,0,0,0,0,0,0,0,0,0,0
3,A2ML1,0.000206,0.001135,0,0,0,0,0,0,0,0,0,0
4,A3GALT2,0.000000,0.003806,0,0,0,0,0,0,0,0,0,0


# more marker sets

In [7]:
mdf = pd.read_csv('../data/gene_sets/Ianevski_2022_markers.csv')

query = [
    'Fibroblasts',
    'Myofibroblasts',
    'Myoblasts',
    'Myocytes',
    'Myoepithelial cells',   
]

mdf = mdf[mdf['Cell type'].isin(query)]
mdf = mdf[mdf['Tissue'] != 'Teeth']
mdf

,Tissue,Cell type,Marker genes,Negative markers
74,Eye,Fibroblasts,"PAX6,COL1A1,TGFB1,ACTA2,ICAM1,FBLN1,CRHBP,FN1,...",NaN
128,Lung,Fibroblasts,"COL3A1,COL5A2,DPT,FN1,GSN,LRP1,PDGFRA,TCF21",NaN
166,Muscle,Myoblasts,"ACHE,ADAM12,ANGPT2,ARL4D,BRAF,CAPN1,CCL2,CDH15...",NaN
167,Muscle,Myocytes,"ACTA1,ACTC1,ACTN2,ACTN3,ADAM12,ANKRD1,ARHGAP26...",NaN
168,Muscle,Myoepithelial cells,"ACTA2,ACTG2,CD109,CDH3,CNN1,EGFR,FST,GRWD1,KRT...",NaN
169,Muscle,Myofibroblasts,"ACTA2,CALD1,CDH11,DES,GFAP,MYL9,PALLD,TAGLN,TNS1",NaN
191,Heart,Fibroblasts,"DCN,GSN,PDGFRA,COL1A1,C7,LUM,Tcf21,Pdgfra",NaN


In [8]:
for idx, row in mdf.iterrows():
    cell_type = row['Cell type'].lower()
    tissue = row['Tissue'].lower()
    genes = row['Marker genes'].split(",")

    column_name = f"ianevski_2022_{tissue}_{cell_type}"

    gx[column_name] = np.where(gx['gene_name'].isin(genes), 1, 0)

gx.head()

,gene_name,2015_mean_tpm,2018_mean_tpm,RNA_polymerase,basal_TFs,cell_cycle,circadian_rhythm,circadian_entrainment,fibroblast_pangloa_db,myogenic_pangloa_db,G2_marker_gene,S_marker_gene,pip_fucci_gene,ianevski_2022_eye_fibroblasts,ianevski_2022_lung_fibroblasts,ianevski_2022_muscle_myoblasts,ianevski_2022_muscle_myocytes,ianevski_2022_muscle_myoepithelial cells,ianevski_2022_muscle_myofibroblasts,ianevski_2022_heart_fibroblasts
0,A1BG,0.077728,0.148410,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A1CF,0.000428,0.001135,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A2M,0.000813,6.078858,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A2ML1,0.000206,0.001135,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A3GALT2,0.000000,0.003806,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
outpath = "../data/gene_sets/gene_annotations.csv"
gx.to_csv(outpath, index=False)
gx.head()

,gene_name,2015_mean_tpm,2018_mean_tpm,RNA_polymerase,basal_TFs,cell_cycle,circadian_rhythm,circadian_entrainment,fibroblast_pangloa_db,myogenic_pangloa_db,G2_marker_gene,S_marker_gene,pip_fucci_gene,ianevski_2022_eye_fibroblasts,ianevski_2022_lung_fibroblasts,ianevski_2022_muscle_myoblasts,ianevski_2022_muscle_myocytes,ianevski_2022_muscle_myoepithelial cells,ianevski_2022_muscle_myofibroblasts,ianevski_2022_heart_fibroblasts
0,A1BG,0.077728,0.148410,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A1CF,0.000428,0.001135,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A2M,0.000813,6.078858,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A2ML1,0.000206,0.001135,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A3GALT2,0.000000,0.003806,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
gx = df.groupby(['gene_name','dataset'])['tpm'].mean().reset_index(drop=False)

sns.histplot(data=gx, 
             x='tpm',
             bins=31,
             hue='dataset',
             log_scale=True)

plt.ylabel('n genes')
plt.xlabel('TPM')
sns.despine()
gx.head()

In [ ]:
tmp = gx[gx['tpm'] > 0.1]
tmp['gene_name'].nunique()